In [14]:
import pandas as pd
import numpy as np

In [15]:
df = pd.read_csv("./data/simple_preprocessed.csv")
df.describe()

,point,timestamp,count
count,3.550113e+06,3.550113e+06,3.550113e+06
mean,3.498905e+03,5.001823e+03,2.334860e+00
std,1.953676e+03,2.688499e+03,4.190819e+00
min,1.000000e+00,0.000000e+00,1.000000e+00
25%,1.869000e+03,2.708000e+03,1.000000e+00
50%,3.575000e+03,5.265000e+03,1.000000e+00
75%,5.159000e+03,7.275000e+03,2.000000e+00
max,7.036000e+03,9.503000e+03,6.000000e+02


In [16]:
count_type = np.int16
df["count"] = df["count"].astype(count_type)

In [17]:
start_hour = pd.read_csv("./data/preprocessed.csv")["timestamp"].min()
test_df = pd.read_csv("./data/source/test.csv")
points_df = pd.read_csv("./data/points_df.csv")
test_df.drop(columns=["lat", "lon", "error"], inplace=True)
test_df["hour"] -= start_hour
test_df["hour"] /= 60 * 60
test_df["hour"] = test_df["hour"].astype(int)
test_df = test_df.merge(points_df, left_on="point", right_on="initial_point")
test_df.drop(columns=["point", "initial_point"], inplace=True)
test_df.rename(columns={"hour": "timestamp", "new_point": "point", "sum": "count"},inplace=True)
test_df["count"] = test_df["count"].astype(count_type)
test_data_per_point = {point: data.drop(columns=["point"]) for point, data in  test_df.groupby("point")}
print(len(test_data_per_point))
test_data_per_point[917]

149


,timestamp,count
400,9876,5
401,10072,14
402,9731,8
403,10116,5
404,9635,5
405,9609,7
406,9681,5
407,10045,8
408,9549,5
409,9734,5


In [18]:
test_df.describe()

,timestamp,count,point
count,700.000000,700.000000,700.000000
mean,9846.421429,9.682857,3663.240000
std,199.468710,6.430390,1897.835651
min,9510.000000,5.000000,214.000000
25%,9682.000000,5.000000,2122.000000
50%,9844.500000,7.000000,4191.000000
75%,10025.000000,11.000000,5238.000000
max,10197.000000,40.000000,6843.000000


In [19]:
data_per_point = {point: data.drop(columns=["point"]) for point, data in df.groupby("point") if point in test_data_per_point}
len(data_per_point)

147

In [20]:
data_per_point[917]

,timestamp,count
412096,1,1
412097,2,1
412098,5,1
412099,8,1
412100,10,2
...,...,...
418614,9499,11
418615,9500,4
418616,9501,5
418617,9502,1


In [21]:
total_hours = df["timestamp"].values.ptp() + 1
print(total_hours)
def transform_to_full_time(point_df):
    """
    Transform sparce dataframe with number of posts at exact hour into
    a list with hours as indices.
    """
    data = np.zeros(total_hours, dtype=count_type)
    for time, count in point_df.values:
        data[time] = count
    return data

9504


In [22]:
data_per_point = {point: transform_to_full_time(data) for point, data in data_per_point.items()}

In [23]:
data_per_point[917].sum()

18816

In [24]:
lag = 30 * 24
roll = 30 * 24
hidden = 100
batch = 10
epochs = 25

In [25]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

def timeline_to_windows(data):
    n = data.shape[0]
    return np.array([data[i: i + lag + roll] for i in range(n - (lag + roll))])

def target_loss(prediction, ground_truth, prediction_start):
    s = 0
    for hour, true_count in ground_truth.values:
        predicted_count = prediction[hour - prediction_start]
        if predicted_count == 0:
            s += 0 if true_count == 0 else 1
        else:
            s += abs(predicted_count - true_count) / predicted_count
    return s

def train_validate(train, test, data):
    X, y = train[:, :lag], train[:, lag:]
    X = X.reshape(X.shape[0], 1, X.shape[1])
    model = Sequential()

    model.add(LSTM(hidden, input_shape=(X.shape[1], X.shape[2]), dropout=0.5))
    model.add(Dense(y.shape[1], activation='relu'))
    model.compile(loss='mean_squared_error')
    model.fit(X, y, validation_split=0.2, epochs=epochs, batch_size=batch, verbose=0, shuffle=True)

    X = data[-lag:]
    X = X.reshape(1, 1, len(X))
    forecast = model.predict(X, batch_size=batch, verbose=0).reshape(-1)
    forecast = list(map(round, forecast))
    return target_loss(forecast, test, len(data))

In [26]:
loss = 0
points = 0
for point, test in test_data_per_point.items():
    if point not in data_per_point:
        print(f"Point {point} is not found in train data!!!")
        continue
    points += test.shape[0]
    data = data_per_point[point]
    windows = timeline_to_windows(data)
    cur_loss = train_validate(windows, test, data)
    loss += cur_loss
    print("Predict point:", point, "Loss:", cur_loss / test.shape[0], "Average posts per hour:", np.mean(data), "Test points:", test.shape[0])
print("Total loss: ", loss / points)

Predict point: 214 Loss: 4.0 Average posts per hour: 0.8745791245791246 Test points: 1
Predict point: 296 Loss: 0.9791666666666667 Average posts per hour: 2.768097643097643 Test points: 4
Predict point: 329 Loss: 5.0 Average posts per hour: 0.9394991582491582 Test points: 4
Predict point: 431 Loss: 0.25 Average posts per hour: 2.310395622895623 Test points: 1
Predict point: 449 Loss: 6.416666666666667 Average posts per hour: 2.0467171717171717 Test points: 18
Predict point: 566 Loss: 0.6666666666666666 Average posts per hour: 1.4433922558922558 Test points: 1
Predict point: 700 Loss: 1.3333333333333333 Average posts per hour: 0.9214015151515151 Test points: 1
Predict point: 702 Loss: 7.5 Average posts per hour: 1.364057239057239 Test points: 1
Predict point: 715 Loss: 0.34767531685215175 Average posts per hour: 7.4602272727272725 Test points: 17
Predict point: 722 Loss: 2.75 Average posts per hour: 1.0710227272727273 Test points: 2
Predict point: 817 Loss: 3.5 Average posts per hour: 1